# Mini-exercice (obligatoire) — Ablation LSTM vs GRU (2 epochs)

Objectif : comparer **LSTM vs GRU** à architecture comparable, sur **2 epochs** uniquement (juste pour observer l'effet).

On garde :
- même vocabulaire (`MAX_WORDS`)
- même longueur (`MAX_LEN`)
- même embedding dim
- mêmes hyperparams d'optimisation

On compare rapidement : accuracy + ROC-AUC sur validation.


## 1) Imports & Setup

In [ ]:
from pathlib import Path
import sys

import numpy as np
import tensorflow as tf

cwd = Path.cwd().resolve()
PROJECT_DIR = None
for p in [cwd] + list(cwd.parents):
    if (p / 'src').exists():
        PROJECT_DIR = p
        break
if PROJECT_DIR is None:
    raise RuntimeError(f"Could not find project root containing 'src' starting from: {cwd}")
sys.path.insert(0, str(PROJECT_DIR))

from src.text_preprocessing import TextPreprocessor
from src.model_architecture import ModelConfig, build_bilstm_model, build_gru_model

print('Project dir:', PROJECT_DIR)
print('Python:', sys.version)
print('TensorFlow:', tf.__version__)

## 2) Charger IMDB (vectorisé + padding)

In [ ]:
MAX_WORDS = 10_000
MAX_LEN = 200
VAL_SIZE = 5_000

pre = TextPreprocessor(max_words=MAX_WORDS, max_len=MAX_LEN)
data = pre.load_imdb_text(validation_size=VAL_SIZE, seed=42)

X_train, y_train = data.X_train, data.y_train
X_val, y_val = data.X_val, data.y_val

print('X_train:', X_train.shape, X_train.dtype)
print('X_val  :', X_val.shape, X_val.dtype)

## 3) Config commune

In [ ]:
cfg = ModelConfig(
    vocab_size=MAX_WORDS,
    max_len=MAX_LEN,
    embedding_dim=128,
    rnn_units=64,
    dropout=0.3,
)
cfg

## 4) Train 2 epochs — BiLSTM

In [ ]:
bilstm = build_bilstm_model(cfg)
hist_bilstm = bilstm.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=2,
    batch_size=128,
    verbose=1,
)
bilstm_metrics = bilstm.evaluate(X_val, y_val, verbose=0)
print('BiLSTM val:', dict(zip(bilstm.metrics_names, [float(x) for x in bilstm_metrics])))

## 5) Train 2 epochs — BiGRU

In [ ]:
bigru = build_gru_model(cfg)
hist_bigru = bigru.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=2,
    batch_size=128,
    verbose=1,
)
bigru_metrics = bigru.evaluate(X_val, y_val, verbose=0)
print('BiGRU val:', dict(zip(bigru.metrics_names, [float(x) for x in bigru_metrics])))

## 6) Comparaison simple

In [ ]:
def last_val(metric_name: str, hist) -> float:
    key = f'val_{metric_name}'
    return float(hist.history.get(key, [np.nan])[-1])

summary = {
    'BiLSTM': {
        'val_accuracy': last_val('accuracy', hist_bilstm),
        'val_roc_auc': last_val('roc_auc', hist_bilstm),
    },
    'BiGRU': {
        'val_accuracy': last_val('accuracy', hist_bigru),
        'val_roc_auc': last_val('roc_auc', hist_bigru),
    },
}
summary

✅ Fin mini-exercice.

Note : sur 2 epochs, les résultats peuvent être bruyants. L'objectif est juste de voir une différence de dynamique (vitesse d'apprentissage/stabilité).